In [61]:
import pandas as pd
from sklearn.preprocessing import MinMaxScaler
import numpy as np

# Load the data
train_data = pd.read_csv('Train.csv')
test_data = pd.read_csv('Google_Stock_Price_Test.csv')

# Inspect the data
print(train_data.head())


       Date    Open    High     Low   Close      Volume
0  1/3/2012  325.25  332.83  324.97  663.59   7,380,500
1  1/4/2012  331.27  333.87  329.08  666.45   5,749,400
2  1/5/2012  329.83  330.75  326.89  657.21   6,590,300
3  1/6/2012  328.34  328.77  323.68  648.24   5,405,900
4  1/9/2012  322.04  322.29  309.46  620.76  11,688,800


In [65]:
scaler = MinMaxScaler(feature_range=(0, 1))
training_set = train_data.iloc[:, 1:2].values  # Assuming the stock price is in the 2nd column
training_set_scaled = scaler.fit_transform(training_set)

X_train, y_train = [], []
for i in range(60, len(training_set_scaled)):
    X_train.append(training_set_scaled[i-60:i, 0])
    y_train.append(training_set_scaled[i, 0])
X_train, y_train = np.array(X_train), np.array(y_train)

# Reshape for LSTM input [samples, time steps, features]
X_train = np.reshape(X_train, (X_train.shape[0], X_train.shape[1], 1))

In [66]:
X_train, y_train = [], []
for i in range(60, len(training_set_scaled)):
    X_train.append(training_set_scaled[i-60:i, 0])
    y_train.append(training_set_scaled[i, 0])
X_train, y_train = np.array(X_train), np.array(y_train)

# Reshape for LSTM input [samples, time steps, features]
X_train = np.reshape(X_train, (X_train.shape[0], X_train.shape[1], 1))


In [74]:
X_train

array([[[0.08581368],
        [0.09701243],
        [0.09433366],
        ...,
        [0.07846566],
        [0.08034452],
        [0.08497656]],

       [[0.09701243],
        [0.09433366],
        [0.09156187],
        ...,
        [0.08034452],
        [0.08497656],
        [0.08627874]],

       [[0.09433366],
        [0.09156187],
        [0.07984225],
        ...,
        [0.08497656],
        [0.08627874],
        [0.08471612]],

       ...,

       [[0.92106928],
        [0.92438053],
        [0.93048218],
        ...,
        [0.95475854],
        [0.95204256],
        [0.95163331]],

       [[0.92438053],
        [0.93048218],
        [0.9299055 ],
        ...,
        [0.95204256],
        [0.95163331],
        [0.95725128]],

       [[0.93048218],
        [0.9299055 ],
        [0.93113327],
        ...,
        [0.95163331],
        [0.95725128],
        [0.93796041]]])

In [4]:
import boto3
import os

s3 = boto3.client('s3')
bucket_name = 'stockbucket-ece528'

# Save preprocessed data
np.save('X_train.npy', X_train)
np.save('y_train.npy', y_train)

# Upload to S3
s3.upload_file('X_train.npy', bucket_name, 'stock-prediction/X_train.npy')
s3.upload_file('y_train.npy', bucket_name, 'stock-prediction/y_train.npy')


In [5]:
import sagemaker
from sagemaker.tensorflow import TensorFlow

# Define the estimator
role = sagemaker.get_execution_role()
bucket = 'stockbucket-ece528'
output_path = f's3://{bucket}/stock-prediction/output'

estimator = TensorFlow(
    entry_point='train.py',
    role=role,
    instance_count=1,
    instance_type='ml.m5.xlarge',
    framework_version='2.12',
    py_version='py310',
    script_mode=True,
    hyperparameters={'epochs': 50, 'batch_size': 32},
    output_path=output_path
)

# Start the training job
estimator.fit({'training': f's3://{bucket}/stock-prediction/'})



/home/ec2-user/anaconda3/envs/tensorflow2_p310/lib/python3.10/site-packages/pydantic/_internal/_fields.py:172: UserWarning: Field name "json" in "MonitoringDatasetFormat" shadows an attribute in parent "Base"
  warnings.warn(


[12/12/24 04:11:29] INFO     Found credentials from IAM Role:                                   ]8;id=110124;file:///home/ec2-user/anaconda3/envs/tensorflow2_p310/lib/python3.10/site-packages/botocore/credentials.py\credentials.py]8;;\:]8;id=519004;file:///home/ec2-user/anaconda3/envs/tensorflow2_p310/lib/python3.10/site-packages/botocore/credentials.py#1075\1075]8;;\
                             BaseNotebookInstanceEc2InstanceRole                                                   

sagemaker.config INFO - Not applying SDK defaults from location: /etc/xdg/sagemaker/config.yaml
sagemaker.config INFO - Not applying SDK defaults from location: /home/ec2-user/.config/sagemaker/config.yaml


[12/12/24 04:11:30] INFO     SageMaker Python SDK will collect telemetry to help us better  ]8;id=448152;file:///home/ec2-user/anaconda3/envs/tensorflow2_p310/lib/python3.10/site-packages/sagemaker/telemetry/telemetry_logging.py\telemetry_logging.py]8;;\:]8;id=341030;file:///home/ec2-user/anaconda3/envs/tensorflow2_p310/lib/python3.10/site-packages/sagemaker/telemetry/telemetry_logging.py#90\90]8;;\
                             understand our user's needs, diagnose issues, and deliver                             
                             additional features.                                                                  
                             To opt out of telemetry, please disable via TelemetryOptOut                           
                             parameter in SDK defaults config. For more information, refer                         
                             to                                                                                    
                             https://sagemaker.readthedocs.io/en/stable/overview.html#confi                        
                             guring-and-using-defaults-with-the-sagemaker-python-sdk.                              

                    INFO     image_uri is not presented, retrieving image_uri based on            ]8;id=724580;file:///home/ec2-user/anaconda3/envs/tensorflow2_p310/lib/python3.10/site-packages/sagemaker/image_uris.py\image_uris.py]8;;\:]8;id=145735;file:///home/ec2-user/anaconda3/envs/tensorflow2_p310/lib/python3.10/site-packages/sagemaker/image_uris.py#675\675]8;;\
                             instance_type, framework etc.                                                         

                    INFO     image_uri is not presented, retrieving image_uri based on            ]8;id=423112;file:///home/ec2-user/anaconda3/envs/tensorflow2_p310/lib/python3.10/site-packages/sagemaker/image_uris.py\image_uris.py]8;;\:]8;id=667120;file:///home/ec2-user/anaconda3/envs/tensorflow2_p310/lib/python3.10/site-packages/sagemaker/image_uris.py#675\675]8;;\
                             instance_type, framework etc.                                                         

                    INFO     Creating training-job with name:                                       ]8;id=970008;file:///home/ec2-user/anaconda3/envs/tensorflow2_p310/lib/python3.10/site-packages/sagemaker/session.py\session.py]8;;\:]8;id=925324;file:///home/ec2-user/anaconda3/envs/tensorflow2_p310/lib/python3.10/site-packages/sagemaker/session.py#1042\1042]8;;\
                             tensorflow-training-2024-12-12-04-11-30-059                                           

2024-12-12 04:11:31 Starting - Starting the training job...
2024-12-12 04:11:45 Starting - Preparing the instances for training...
2024-12-12 04:12:31 Downloading - Downloading the training image......
2024-12-12 04:13:32 Training - Training image download completed. Training in progress...2024-12-12 04:13:38.705108: I tensorflow/core/platform/cpu_feature_guard.cc:182] This TensorFlow binary is optimized to use available CPU instructions in performance-critical operations.
To enable the following instructions: AVX512F, in other operations, rebuild TensorFlow with the appropriate compiler flags.
2024-12-12 04:13:40,957 sagemaker-training-toolkit INFO     Imported framework sagemaker_tensorflow_container.training
2024-12-12 04:13:40,958 sagemaker-training-toolkit INFO     No GPUs detected (normal if no gpus installed)
2024-12-12 04:13:40,958 sagemaker-training-toolkit INFO     No Neurons detected (normal if no neurons installed)
2024-12-12 04:13:41,350 sagemaker-training-toolkit INFO    

In [7]:
from sagemaker.tensorflow import TensorFlowModel

# Reference the model.tar.gz file
#estimator.latest_training_job.name
model_uri = 's3://stockbucket-ece528/stock-prediction/output/tensorflow-training-2024-12-12-04-11-30-059/output/model.tar.gz'

# Define the TensorFlow model
tf_model = TensorFlowModel(
    model_data=model_uri,
    role=role,
    framework_version='2.12',
    sagemaker_session=sagemaker.Session()
)

# Deploy the model
predictor = tf_model.deploy(
    initial_instance_count=1,
    instance_type='ml.m5.large'
)


[12/12/24 04:22:11] INFO     image_uri is not presented, retrieving image_uri based on instance_type,  ]8;id=225360;file:///home/ec2-user/anaconda3/envs/tensorflow2_p310/lib/python3.10/site-packages/sagemaker/tensorflow/model.py\model.py]8;;\:]8;id=112840;file:///home/ec2-user/anaconda3/envs/tensorflow2_p310/lib/python3.10/site-packages/sagemaker/tensorflow/model.py#508\508]8;;\
                             framework etc.                                                                        

[12/12/24 04:22:12] INFO     Creating model with name: tensorflow-inference-2024-12-12-04-22-12-004 ]8;id=643018;file:///home/ec2-user/anaconda3/envs/tensorflow2_p310/lib/python3.10/site-packages/sagemaker/session.py\session.py]8;;\:]8;id=752851;file:///home/ec2-user/anaconda3/envs/tensorflow2_p310/lib/python3.10/site-packages/sagemaker/session.py#4094\4094]8;;\

                    INFO     Creating endpoint-config with name                                     ]8;id=367372;file:///home/ec2-user/anaconda3/envs/tensorflow2_p310/lib/python3.10/site-packages/sagemaker/session.py\session.py]8;;\:]8;id=619693;file:///home/ec2-user/anaconda3/envs/tensorflow2_p310/lib/python3.10/site-packages/sagemaker/session.py#5889\5889]8;;\
                             tensorflow-inference-2024-12-12-04-22-12-584                                          

                    INFO     Creating endpoint with name                                            ]8;id=491783;file:///home/ec2-user/anaconda3/envs/tensorflow2_p310/lib/python3.10/site-packages/sagemaker/session.py\session.py]8;;\:]8;id=222014;file:///home/ec2-user/anaconda3/envs/tensorflow2_p310/lib/python3.10/site-packages/sagemaker/session.py#4711\4711]8;;\
                             tensorflow-inference-2024-12-12-04-22-12-584                                          

-----!

In [77]:
import boto3
import numpy as np
import pandas as pd
from sklearn.preprocessing import MinMaxScaler
import json

# Initialize SageMaker runtime client
runtime_client = boto3.client('sagemaker-runtime')

# Load your data (CSV or DataFrame)
df = pd.read_csv('AAPL.csv', parse_dates=['Date'], dayfirst=True)

# Preprocess the data: Select 'Close' price and scale it
training_set = df['Close'].values.reshape(-1, 1)
scaler = MinMaxScaler(feature_range=(0, 1))
training_set_scaled = scaler.fit_transform(training_set)

# Prepare the last 60 days of data for prediction
new_data = df[['Close']].tail(60).values
new_data_scaled = scaler.transform(new_data)
new_data_reshaped = np.reshape(new_data_scaled, (1, 60, 1))

# Convert the input to a format expected by the model (JSON)
payload = json.dumps(new_data_reshaped.tolist())

# Set the endpoint name
endpoint_name = 'tensorflow-inference-2024-12-12-04-22-12-584'

# Function to predict the next n days
def predict_next_n_days(n):
    predictions = []
    current_input = new_data_scaled[-60:].reshape(1, 60, 1)  # Last 60 days as input
    
    for _ in range(n):
        # Invoke the SageMaker endpoint for the next day
        response = runtime_client.invoke_endpoint(
            EndpointName=endpoint_name,
            ContentType='application/json',
            Body=json.dumps(current_input.tolist())
        )

        # Extract the prediction from the response
        prediction = json.loads(response['Body'].read().decode('utf-8'))
        
        # Reverse the scaling to get the original predicted price
        predicted_price = scaler.inverse_transform(np.array(prediction['predictions']).reshape(-1, 1))
        predictions.append(predicted_price[0][0])
        
        # Update current_input by adding the predicted value and removing the oldest data point
        current_input = np.append(current_input[:, 1:, :], [[predicted_price[-1]]], axis=1)
    
    return predictions

# Predict the next 6 days' prices
predicted_prices = predict_next_n_days(6)

# Output the predicted prices for the next 6 days
for i, price in enumerate(predicted_prices, 1):
    print(f"Predicted price for day {i}: {price}")


Predicted price for day 1: 203.12845922064
Predicted price for day 2: 269.22138965599993
Predicted price for day 3: 308.8195415296
Predicted price for day 4: 325.60044385439994
Predicted price for day 5: 333.43366546719994
Predicted price for day 6: 337.5151335951999
